In [46]:
import sys
sys.path.append("../")

In [47]:
from utils.common_train_utils import *
from utils.draw_candle_image import *
from utils.evaluate_old_models import *
from functools import partial
import os
from tabulate import tabulate
import random
import warnings
warnings.filterwarnings("ignore")


In [48]:
def print_loss_of_all_models_in_dataset_folder(folder_object):
    dataset_path = folder_object.path
    
    config = read_config(dataset_path)
    candle_type_and_directory_save: dict = config["candle_type_and_directory_save"]
    days_result = config["days_result"]
    
    dataset_test = load_dataset_of_each_type_and_combine(f"{dataset_path}/test", candle_type_and_directory_save)
    
    batch_size = random.randint(1, int(len(dataset_test) / 4))
    shuffle_buffer = random.randint(1, int(len(dataset_test) / 3))
    
    dataset_test_1 = dataset_test.shuffle(shuffle_buffer)
    dataset_test_2 = dataset_test_1.batch(batch_size)
    
    # Ví dụ
    all_models_folder_path = os.path.join(os.path.abspath(dataset_path), "model_save")
    df = evaluate_all_models_in_dataset_folder(
        all_models_folder_path, 
        dataset_test_2,
        days_result
    )
    
    # print(f"----------------- {folder_name} -----------------")
    # print(df)
    
    df["total_records"] = len(dataset_test)
    df["batch_size"] = batch_size
    df["shuffle_buffer"] = shuffle_buffer
    
    return df

In [49]:
dataset_parent_folder = os.path.abspath("../dataset")

In [50]:
just_symbol = input("symbol = (null is all folder)").lower().strip()

In [51]:
all_dfs = {}
for f in os.scandir(dataset_parent_folder):
    if not f.is_dir(): continue
    if just_symbol and f.name != f"{just_symbol}_with_ema_macd_trend": continue
    
    df = print_loss_of_all_models_in_dataset_folder(f)
    all_dfs[f.name] = df

15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - loss: 6.5505 - mae: 1.7306 - mse: 6.5505
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - loss: 6.5851 - mae: 1.7403 - mse: 6.5851
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - loss: 6.5227 - mae: 1.6964 - mse: 6.5227


In [52]:
list(all_dfs.keys())

['eth_with_ema_macd_trend']

In [53]:
for dataset_name in all_dfs:
    print(dataset_name)
    print(tabulate(all_dfs[dataset_name], headers='keys', tablefmt='pretty'))
    print("\n")

eth_with_ema_macd_trend
+---+------------------------------------------------------+-------------------+--------------------+---------------+------------+----------------+
|   |                      model_name                      |        mse        |        mae         | total_records | batch_size | shuffle_buffer |
+---+------------------------------------------------------+-------------------+--------------------+---------------+------------+----------------+
| 2 |          model_5_biLSTM_ema_macd_trend_1_1           | 7.014735698699951 | 1.7484924793243408 |      231      |     16     |       22       |
| 1 | channels_to_time_step_attention_ema_macd_trend_1_1_1 | 7.06008768081665  | 1.7850439548492432 |      231      |     16     |       22       |
| 0 |   channels_to_time_step_attention_ema_macd_trend_1   | 7.100112438201904 | 1.7885512113571167 |      231      |     16     |       22       |
+---+------------------------------------------------------+-------------------+--------

In [54]:
#                model_5_biLSTM_ema_macd_trend_1_1_1_1_1_1    
            #  channels_to_time_step_attention_ema_macd_trend 